In [17]:
!pip install numpy==1.23.5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import numpy as np

from sklearn import datasets

# Diabetes

In [2]:
diabetes = datasets.load_diabetes()
X, y = diabetes.data, diabetes.target

In [3]:
all_data = [{0:[X,y]}]


# California housing

In [4]:
from sklearn.datasets import fetch_california_housing

X, y = fetch_california_housing(return_X_y=True, as_frame=True)
all_data.append({0:[X.to_numpy(),y.to_numpy()]})

# Liver disorders

In [5]:
from sklearn.datasets import fetch_openml
ld = fetch_openml(name='liver-disorders')

In [7]:
X, y = ld['data'].to_numpy(),ld['target'].to_numpy()
all_data.append({0:[X,y]})

In [8]:

from sklearn.model_selection import train_test_split
dict_data = {}
dict_data["Diabetes"] = {}
dict_data["California housing"] = {}
dict_data["Liver disorders"] = {}

for k in all_data[0]:
    x01,x02,y01,y02 = train_test_split(all_data[0][k][0], all_data[0][k][1], test_size=0.3,random_state=42)
    dict_data["Diabetes"][k] = {"train":{"X":x01,"y":y01},"test":{"X":x02,"y":y02}}
    
for k in all_data[1]:
    x11,x12,y11,y12 = train_test_split(all_data[1][k][0], all_data[1][k][1],test_size=0.3,random_state=42)
    dict_data["California housing"][k] = {"train":{"X":x11,"y":y11},"test":{"X":x12,"y":y12}} 

for k in all_data[2]:
    x11,x12,y11,y12 = train_test_split(all_data[2][k][0], all_data[2][k][1],test_size=0.3,random_state=42)
    dict_data["Liver disorders"][k] = {"train":{"X":x11,"y":y11},"test":{"X":x12,"y":y12}} 

all_data = dict_data

# Boosting

In [9]:
import xgboost as xgb
from boosted_forest import CascadeBoostingRegressor
from deepforest import CascadeForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import optuna

from sklearn.model_selection import KFold
#xgb.set_config(verbosity=2)

def make_modelXGB(max_depth,layers,C):
    return xgb.XGBRegressor(max_depth = max_depth, n_estimators = layers)

def make_modelCascade(max_depth,layers,C):
    return CascadeForestRegressor(max_depth = max_depth, max_layers = layers, n_estimators=4)

def make_modelBoosted(max_depth,layers,C):
    return CascadeBoostingRegressor(C=C, n_layers=layers, n_estimators = 1, max_depth=max_depth, n_iter_no_change = 1, validation_fraction = 0.1, learning_rate = 0.9)


models = {"XGB":make_modelXGB,"Cascade Forest":make_modelCascade, "Boosted Forest": make_modelBoosted}

bo_data = []    

for model_name in models:
    make_model = models[model_name]
    for ds_name in all_data:
        for depth in all_data[ds_name]:
            dat = all_data[ds_name][depth]
            x_train = dat["train"]["X"]
            x_test = dat["test"]["X"]
            Y_train = dat["train"]["y"].flatten()
            Y_test = dat["test"]["y"].flatten()            

            def objective(trial):
                layers = trial.suggest_int('layers', 5, 15)
                max_depth = trial.suggest_int('max_depth', 1, 2)

                if model_name == "Boosted Forest":
                    C = trial.suggest_int('C', 1, 2000)
                else:
                    C = 0

                kf = KFold(n_splits=3)
                scores = []
                for _, (train_index, test_index) in enumerate(kf.split(x_train)):
                    model = make_model(max_depth,layers,C)
                    
                    model.fit(
                         x_train[train_index],
                         Y_train[train_index],
                    )
                    y_pred = model.predict(x_train[test_index]) #, batch_size=batch_size)
                    scores.append(mean_squared_error(Y_train[test_index].flatten(),y_pred.flatten()))
                return np.asarray(scores).mean() 
            
            study = optuna.create_study(direction='minimize')
            study.optimize(objective, n_trials=20)    
            
            layers = study.best_trial.params["layers"]  
            max_depth = study.best_trial.params["max_depth"]  

            if model_name == "Boosted Forest":
                C = study.best_trial.params["C"]  
            else:
                C = 0
            model = make_model(max_depth,layers,C)
            model.fit(
                 x_train,
                 Y_train,
            )        
            
            y_pred = model.predict(x_test) #, batch_size=batch_size)
            mse_score = mean_squared_error(Y_test.flatten(),y_pred.flatten())
            mae_score = mean_absolute_error(Y_test.flatten(),y_pred.flatten())
            print(model_name,ds_name,depth,mse_score, mae_score, Y_test.min(),Y_test.max())     
            bo_data.append([model_name,ds_name,depth,mse_score, mae_score])
    

/home/keen/.pyenv/versions/3.9.13/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-07-01 21:48:57,363] A new study created in memory with name: no-name-607ca942-3791-4824-8531-5cb7ac50eed1
[I 2024-07-01 21:48:57,379] Trial 0 finished with value: 3552.049809300428 and parameters: {'layers': 9, 'max_depth': 2}. Best is trial 0 with value: 3552.049809300428.
[I 2024-07-01 21:48:57,390] Trial 1 finished with value: 3974.1972731973124 and parameters: {'layers': 5, 'max_depth': 1}. Best is trial 0 with value: 3552.049809300428.
[I 2024-07-01 21:48:57,405] Trial 2 finished with value: 3551.1328837323167 and parameters: {'layers': 15, 'max_depth': 2}. Best is trial 2 with value: 3551.1328837323167.
[I 2024-07-01 21:48:57,418] Trial 3 finished with value: 3562.682925637481 and parameters: {'layers': 8, 'ma

XGB Diabetes 0 2817.1593648895614 42.626976873641624 37.0 310.0


[I 2024-07-01 21:48:57,921] Trial 8 finished with value: 0.7329674326935539 and parameters: {'layers': 7, 'max_depth': 1}. Best is trial 2 with value: 0.472665618321084.
[I 2024-07-01 21:48:57,948] Trial 9 finished with value: 0.708997527531963 and parameters: {'layers': 8, 'max_depth': 1}. Best is trial 2 with value: 0.472665618321084.
[I 2024-07-01 21:48:57,982] Trial 10 finished with value: 0.4317159336020802 and parameters: {'layers': 14, 'max_depth': 2}. Best is trial 10 with value: 0.4317159336020802.
[I 2024-07-01 21:48:58,015] Trial 11 finished with value: 0.4317159336020802 and parameters: {'layers': 14, 'max_depth': 2}. Best is trial 10 with value: 0.4317159336020802.
[I 2024-07-01 21:48:58,048] Trial 12 finished with value: 0.42023470445161254 and parameters: {'layers': 15, 'max_depth': 2}. Best is trial 12 with value: 0.42023470445161254.
[I 2024-07-01 21:48:58,081] Trial 13 finished with value: 0.42023470445161254 and parameters: {'layers': 15, 'max_depth': 2}. Best is tri

XGB California housing 0 0.43724526678809283 0.480957898873485 0.14999 5.00001


[I 2024-07-01 21:48:58,517] Trial 13 finished with value: 9.142864334793975 and parameters: {'layers': 14, 'max_depth': 2}. Best is trial 13 with value: 9.142864334793975.
[I 2024-07-01 21:48:58,537] Trial 14 finished with value: 9.20757607228616 and parameters: {'layers': 12, 'max_depth': 2}. Best is trial 13 with value: 9.142864334793975.
[I 2024-07-01 21:48:58,558] Trial 15 finished with value: 9.22612365133614 and parameters: {'layers': 15, 'max_depth': 2}. Best is trial 13 with value: 9.142864334793975.
[I 2024-07-01 21:48:58,580] Trial 16 finished with value: 9.159063491760769 and parameters: {'layers': 13, 'max_depth': 2}. Best is trial 13 with value: 9.142864334793975.
[I 2024-07-01 21:48:58,600] Trial 17 finished with value: 9.159063491760769 and parameters: {'layers': 13, 'max_depth': 2}. Best is trial 13 with value: 9.142864334793975.
[I 2024-07-01 21:48:58,620] Trial 18 finished with value: 9.142864334793975 and parameters: {'layers': 14, 'max_depth': 2}. Best is trial 13 w

...

Boosted Forest Diabetes 0 2739.866116146446 42.389746197634615 37.0 310.0


[I 2024-07-01 22:21:09,295] Trial 0 finished with value: 0.25537992590979247 and parameters: {'layers': 13, 'max_depth': 2, 'C': 1304}. Best is trial 0 with value: 0.25537992590979247.
[I 2024-07-01 22:30:10,298] Trial 1 finished with value: 0.3086456121878949 and parameters: {'layers': 14, 'max_depth': 1, 'C': 1290}. Best is trial 0 with value: 0.25537992590979247.
[I 2024-07-01 22:38:57,746] Trial 2 finished with value: 0.2595362936081589 and parameters: {'layers': 10, 'max_depth': 2, 'C': 461}. Best is trial 0 with value: 0.25537992590979247.
[I 2024-07-01 22:47:17,593] Trial 3 finished with value: 0.2547427038254455 and parameters: {'layers': 15, 'max_depth': 2, 'C': 3}. Best is trial 3 with value: 0.2547427038254455.
[I 2024-07-01 22:50:28,054] Trial 4 finished with value: 0.4561657058409064 and parameters: {'layers': 6, 'max_depth': 1, 'C': 1309}. Best is trial 3 with value: 0.2547427038254455.
[I 2024-07-01 22:54:19,368] Trial 5 finished with value: 0.4302170984287476 and parame

Boosted Forest California housing 0 0.23910495651027874 0.3291176889841468 0.14999 5.00001


[I 2024-07-02 00:50:40,008] Trial 0 finished with value: 10.159922529560957 and parameters: {'layers': 5, 'max_depth': 1, 'C': 1030}. Best is trial 0 with value: 10.159922529560957.
[I 2024-07-02 00:50:44,961] Trial 1 finished with value: 9.771143483503845 and parameters: {'layers': 13, 'max_depth': 1, 'C': 1969}. Best is trial 1 with value: 9.771143483503845.
[I 2024-07-02 00:50:52,196] Trial 2 finished with value: 9.45590958606107 and parameters: {'layers': 11, 'max_depth': 2, 'C': 1362}. Best is trial 2 with value: 9.45590958606107.
[I 2024-07-02 00:50:57,879] Trial 3 finished with value: 9.082937543252795 and parameters: {'layers': 14, 'max_depth': 2, 'C': 1581}. Best is trial 3 with value: 9.082937543252795.
[I 2024-07-02 00:51:05,639] Trial 4 finished with value: 9.45027485823853 and parameters: {'layers': 6, 'max_depth': 1, 'C': 1960}. Best is trial 3 with value: 9.082937543252795.
[I 2024-07-02 00:51:12,101] Trial 5 finished with value: 9.87204097971951 and parameters: {'layers

Boosted Forest Liver disorders 0 8.202961142766497 2.309401348763574 0.0 15.0
